In [19]:
from Alpaca_keyes import API_KEY, SECRET_KEY
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest

In [20]:
api_key = API_KEY
secret_key = SECRET_KEY

symbol = "TSLA"

stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key)

In [21]:
now = datetime.now(ZoneInfo("America/New_York"))

req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 15, unit = TimeFrameUnit.Minute), # specify timeframe
    start = now - timedelta(days = 50),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    #limit = 2,                                               # specify limit
)

In [22]:
# timestamp is in UTC time
stock_historical_data_client.get_stock_bars(req).df

open      high     low     close  \
symbol timestamp                                                         
TSLA   2024-11-04 16:30:00+00:00  246.0069  246.2600  244.37  244.6701   
       2024-11-04 16:45:00+00:00  244.7200  245.1200  243.93  245.1100   
       2024-11-04 17:00:00+00:00  245.0900  245.4100  243.65  244.1400   
       2024-11-04 17:15:00+00:00  244.1774  244.7800  243.41  244.5880   
       2024-11-04 17:30:00+00:00  244.6800  245.1800  244.25  245.0135   
...                                    ...       ...     ...       ...   
       2024-12-24 15:00:00+00:00  449.4733  451.0000  446.72  448.5200   
       2024-12-24 15:15:00+00:00  448.4650  450.3999  445.42  449.5500   
       2024-12-24 15:30:00+00:00  449.5781  450.2000  447.70  449.8845   
       2024-12-24 15:45:00+00:00  449.9899  452.5500  449.40  451.4500   
       2024-12-24 16:00:00+00:00  451.5250  451.9900  448.80  451.4891   

                                     volume  trade_count        vwap  
symbol timestamp                                                      
TSLA   2024-11-04 16:30:00+00:00  3164079.0      48786.0  245.356211  
       2024-11-04 16:45:00+00:00  2647002.0      49768.0  244.527305  
       2024-11-04 17:00:00+00:00  2085218.0      40621.0  244.503229  
       2024-11-04 17:15:00+00:00  1973573.0      42889.0  244.177265  
       2024-11-04 17:30:00+00:00  1276921.0      36246.0  244.809105  
...                                     ...          ...         ...  
       2024-12-24 15:00:00+00:00  4078514.0      69893.0  448.624298  
       2024-12-24 15:15:00+00:00  3652364.0      63026.0  447.894060  
       2024-12-24 15:30:00+00:00  2451165.0      46888.0  448.985039  
       2024-12-24 15:45:00+00:00  3801635.0      59301.0  451.179233  
       2024-12-24 16:00:00+00:00  2274457.0      38664.0  450.496118  

[2223 rows x 7 columns]

I have 15 min to calculate the missing candle, I need to write a function which converts live datasream into a 15 min candle

In [13]:
import nest_asyncio
nest_asyncio.apply()

async def stock_data_stream_handler(data):
    print(data)

In [15]:
from alpaca.data.live.stock import StockDataStream

stock_data_stream_client = StockDataStream(api_key, secret_key)

symbols = [symbol]

stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

In [23]:
import asyncio
from alpaca.data.live.stock import StockDataStream

stock_data_stream_client = StockDataStream(api_key, secret_key)

async def stock_data_stream_handler(data):
    print(data)

symbols = [symbol]

# Subscribe to quotes and trades
stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

async def run_stream():
    await stock_data_stream_client.run()

async def main():
    # Create a task for running the stream
    stream_task = asyncio.create_task(run_stream())

    # Let the stream run for a specific duration (e.g., 60 seconds)
    await asyncio.sleep(60)

    # Stop the stream client
    await stock_data_stream_client.stop()

    # Wait for the stream task to complete
    await stream_task

# Run the main function
asyncio.run(main())


symbol='TSLA' timestamp=datetime.datetime(2024, 12, 24, 16, 21, 14, 355609, tzinfo=datetime.timezone.utc) bid_price=452.49 bid_size=1.0 bid_exchange='V' ask_price=454.5 ask_size=1.0 ask_exchange='V' conditions=['R'] tape='C'
symbol='TSLA' timestamp=datetime.datetime(2024, 12, 24, 16, 21, 14, 392991, tzinfo=datetime.timezone.utc) exchange='V' price=452.72 size=33.0 id=2779 conditions=['@', 'I'] tape='C'
symbol='TSLA' timestamp=datetime.datetime(2024, 12, 24, 16, 21, 14, 393004, tzinfo=datetime.timezone.utc) exchange='V' price=452.72 size=7.0 id=2780 conditions=['@', 'I'] tape='C'
symbol='TSLA' timestamp=datetime.datetime(2024, 12, 24, 16, 21, 14, 412264, tzinfo=datetime.timezone.utc) bid_price=452.49 bid_size=1.0 bid_exchange='V' ask_price=454.5 ask_size=1.0 ask_exchange='V' conditions=['R'] tape='C'
symbol='TSLA' timestamp=datetime.datetime(2024, 12, 24, 16, 21, 14, 412264, tzinfo=datetime.timezone.utc) exchange='V' price=452.71 size=40.0 id=2781 conditions=['@', 'I'] tape='C'
symbol='

TimeoutError: 